<h1>
<center>
Midterm 3
</center>
</h1>
<div class=h1_cell>
<p>
There are 2 parts on the midterm, 50 points each. Each part has 3 questions.
<p>
We will use the Titanic table for this midterm. I have a wrangled version of it on google sheets that I set up for you.
<p>
</div>

In [0]:
import pandas as pd


titanic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR2XbLnUuYlxaFMdS8_bBX3iKUIDEii6Lg5Rxesf-Oh8a6z8-vAN6UDGejaOrBg5130h4O_dLkecKWQ/pub?output=csv')



In [0]:
pd.set_option('display.max_columns', None)

In [3]:
titanic_table.head(1)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_age,filled_age,emb_C,emb_Q,emb_S,emb_nan,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,ok_child,pclass_1,pclass_2,pclass_3,pclass_nan
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,22.0,0,0,1,0,Child,1,0,0,0,1,0,0,0,1,0


In [4]:
!rm library_w19_week7.py

rm: cannot remove 'library_w19_week7.py': No such file or directory


In [5]:
from google.colab import files
files.upload()

Saving library_w19_week7.py to library_w19_week7.py


{'library_w19_week7.py': b'import pandas as pd\r\nimport numpy as np\r\nfrom functools import reduce\r\nfrom types import SimpleNamespace\r\nimport random\r\nimport matplotlib.pyplot as plt  \r\nimport operator\r\n\r\ndef predictor_case(row, pred, target):\r\n  case_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\r\n  actual = row[target]\r\n  prediction = row[pred]\r\n  case = case_dict[(prediction, actual)]\r\n  return case\r\n\r\ndef informedness(cases):\r\n  tp = 0\r\n  if \'true_positive\' in cases:\r\n    tp = cases[\'true_positive\']\r\n  tn = 0\r\n  if \'true_negative\' in cases:\r\n    tn = cases[\'true_negative\']\r\n  fp = 0\r\n  if \'false_positive\' in cases:\r\n    fp = cases[\'false_positive\']\r\n  fn = 0\r\n  if \'false_negative\' in cases:\r\n    fn = cases[\'false_negative\']\r\n  if (((tp+fn) == 0) or ((tn+fp) == 0)):\r\n    return -1\r\n  else:\r\n    recall = 1.0*tp/(tp+fn)\r\n    specificity = 1.0*

In [6]:
from library_w19_week7 import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 euclidean_distance	 f1	 find_best_splitter	 forest_builder	 
forest_scores	 generate_table	 gig	 gini	 informedness	 k_fold	 k_fold_random	 knn	 knn_tester	 
oob_forest_scores	 oob_test	 oob_vote_taker	 path_id	 predictor_case	 probabilities	 produce_scores	 reorder_paths	 shuffle	 
tree_predictor	 verify_unique	 vote_taker	 


<h2>Columns used in all questions</h2>

We will use 14 columns from the table to make predictions on `Survived`.

In [0]:
usable_columns = [
 'emb_C',
 'emb_Q',
 'emb_S',
 'emb_nan',
 'age_Child',
 'age_Adult',
 'age_Senior',
 'no_age',
 'ok_child',
 'sex_female',
 'pclass_1',
 'pclass_2',
 'pclass_3',
 'pclass_nan'
]

target = 'Survived'

<h1>
Part 1. Explore random forest behavior
</h1>
<div class=h1_cell>
<p>
For prediction purposes, we have been treating a forest as a single predictor. It does call on its trees to get a prediction, but we don't see that. All we see is the final prediction. I'd like to dig deeper into individual tree behavior. I'll ask you to complete a set of programming problems to do this.
</div>

<h2>
Problem 1. Build a matrix of tree predictions (5 points)
</h2>
<div class=h1_cell>
<p>
The matrix will be a list of lists. Let's say the forest we are working with has 13 trees. Then the first list will be the predictions of the 13 trees for row 0 of the Titanic table, i.e., it will be a list of 13 binary values. The next list will be the 13 predictions for row 1 of the Titanic table. Given that we have 891 rows in the Titanic table, we will have 891 lists in our outer list.
<p>
Let's say I store my matrix in `all_trees`. Then `all_trees[i][j]` will represent the vote of the jth tree for row i.
  <p>
    Here is what you should see for the first 10 rows.
<p>
<pre>
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]]
 </pre>

</div>

<h2>
Here is forest to test on
</h2>
<div class=h1_cell>
<p>
  But first set random seeds so you get same results as mine.
</div>

In [0]:
import random

rng = np.random.RandomState(24)  #Will pass as arg to pandas sample method
random.seed(2000)

In [9]:
forest13 = forest_builder(titanic_table, usable_columns, target, hypers={'total-trees':13, 'random-state':rng})
len(forest13)

13


Build the voting matrix for the 13 trees. 

In [11]:
all_trees = []
for index, row in titanic_table.iterrows():
  temp_list = []
  for tree in forest13:
    prediction = tree_predictor(row,tree)
    temp_list.append(prediction)
  all_trees.append(temp_list)
  
all_trees[:10]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]]

<h1>
Problem 2: most incorrect tree (15 points)
</h1>
<div class=h1_cell>
<p>
Produce an ordered list, from most incorrect to least incorrect, of the 13 trees taken over the 891 rows. A tree is incorrect if its prediction is a mismatch with the `Survived` column in the target row. Here are my results.
  <pre>
[(9, 317),
 (8, 314),
 (5, 310),
 (12, 304),
 (0, 296),
 (11, 296),
 (3, 278),
 (7, 254),
 (6, 248),
 (4, 217),
 (1, 190),
 (10, 190),
 (2, 184)]
 </pre>
  The loser is tree9. It was incorrect 317 out of 891 times.
</div>

In [13]:
len(titanic_table)

891

In [22]:
tree_a = []
for index, row in titanic_table.iterrows():
  target = row['Survived']
  for tree in forest13:
    prediction = tree_predictor(row,tree)
    if (prediction == row['Survived']):
      tree_a.append(forest13.index(tree))
all_val = []
for item in range(len(forest13)):
  all_val.append((item,len(titanic_table)-tree_a.count(item)))
  all_val.sort(key=operator.itemgetter(1),reverse=True)
print(all_val)

[(9, 317), (8, 314), (5, 310), (12, 304), (0, 296), (11, 296), (3, 278), (7, 254), (6, 248), (4, 217), (1, 190), (10, 190), (2, 184)]


<h1>
Problem 3: voting blocks (30 points)
</h1>
<div class=h1_cell>
<p>
Taking trees as triples, produce an ordered list of which trees vote the same (right or wrong) most often.
  <p>
    For each specific row:
    <pre>
 1. if (tree1,tree2, tree3) vote the same then I record that.
 2. if (tree1,tree2, tree4) vote the same then I record that.
 etc
</pre>
I do that for the entire table. Here is first part of sorted list I got for forest13.
<pre>
[((5, 8, 12), 849),
 ((2, 7, 10), 787),
 ((2, 3, 10), 769),
 ((1, 4, 6), 742),
 ((3, 7, 10), 721),
 ((1, 2, 10), 719),
 ((2, 3, 7), 693),
 ((2, 6, 10), 689),
 ((7, 9, 10), 679),
 ((1, 6, 10), 676)]
</pre>
  So tree5 and tree8 and tree12 vote the same (right or wrong) 849 out of 891 times.
</div>

In [0]:
list_columns = []
for i in range(len(all_trees[0])):
  tree_list = [row[i] for row in all_trees]
  list_columns.append(tree_list)

In [37]:
def vote_same(tree1,tree2,tree3):
  count = 0
  for row in range(len(tree1)):
    if (tree1[row] == tree2[row] == tree3[row]):
      indicator = True
      count += 1
    else:
      indicator = False
  return count

def triples(alist):
  result = []
  for p1 in range(len(alist)):
    for p2 in range(p1+1,len(alist)):
      for p3 in range(p2+1, len(alist)):
        result.append([(p1,alist[p1]),(p2,alist[p2]),(p3,alist[p3])])
  return result

l_pairs = triples(list_columns)
result = []
for pair in l_pairs:
  count = vote_same(pair[0][1],pair[1][1],pair[2][1])
  i_tup = (pair[0][0],pair[1][0],pair[2][0])
  vote_tup = (i_tup,count)
  result.append(vote_tup)
result.sort(key= operator.itemgetter(1),reverse=True)
result[:10]

[((5, 8, 12), 849),
 ((2, 7, 10), 787),
 ((2, 3, 10), 769),
 ((1, 4, 6), 742),
 ((3, 7, 10), 721),
 ((1, 2, 10), 719),
 ((2, 3, 7), 693),
 ((2, 6, 10), 689),
 ((7, 9, 10), 679),
 ((1, 6, 10), 676)]

In [0]:

block3_list[:10]

[((5, 8, 12), 849),
 ((2, 7, 10), 787),
 ((2, 3, 10), 769),
 ((1, 4, 6), 742),
 ((3, 7, 10), 721),
 ((1, 2, 10), 719),
 ((2, 3, 7), 693),
 ((2, 6, 10), 689),
 ((7, 9, 10), 679),
 ((1, 6, 10), 676)]

It looks like for 891 rows, trees 5 and 8 and 12 vote the same 849 times.

<h1>
Part 2. Explore knn behavior
</h1>
<div class=h1_cell>
<p>
If I set k=10, then I take the 10 rows with the smallest distance as voters. You all know that. But the value for k is somewhat arbitrary. What if I chose k=11? Would it matter?
  <p>
So what I am interested in is if I set k=10, what about the 11th row? It is the one that just missed the cut. Maybe it has useful info to offer but I keep cutting it off. I am going to ask you to explore the set of just left out rows, e.g., the 11th row when k=10. If we find that the 11th row is usually correct, we can increment k to be 11.
    <p>
      I'll ask you to do exploration in a set of 3 problems.
</div>

In [0]:
small_titanic = titanic_table[:100]  #use this slice in your code to cut down on computing time

<h1>
Problem 4: A row's just-missed counts (15 points)
</h1>
<div class=h1_cell>
<p>
  Loop through all the rows in the small titanic table and compute the sorted list of differences for each. For now, I don't care about the first k. All I want is the index of the k+1 item in the sorted distances, i.e., the row that almost made the cut but not quite.
For instance, if k is 10, I want to know the index of 11th item (actually the 10th item with 0-indexing). Record the 11th item's index. When finished print out how many times each row was just missed. Here are the first part of my results. I will use k=10 for all the questions in this part.
  <pre>
[(91, 11),
 (51, 8),
 (2, 7),
 (76, 7),
 (5, 6),
 (21, 6),
 (64, 5),
 (17, 4),
 (23, 4),
 (26, 4)]
 </pre>
  So row 51 just missed being in the top-10 8 times.
</div>

In [68]:
def knn2(row_index,table,columns,k,target):
  table2 = table.copy()
  distances = []
  table2 = table.as_matrix(columns=columns)
  row1 = table2[row_index]
  for item in range(len(table)):
    if item == row_index:
      continue
    distances.append((item,euclidean_distance(row1,table2[item])))
  distances.sort(key=operator.itemgetter(1))
  
  return distances[k]  

def knn_tester2(table,k,columns, target):
  result = []
  count_dict = {}
  for i in range(len(small_titanic)):
    count_dict[i] = 0
  
  for n, row in small_titanic.iterrows():
    topk = [knn2(n,small_titanic,usable_columns,10,target)]
    for item in topk:
      count_dict[item[0]] +=1
  

  for index in range(len(small_titanic)):
    tup = (index,count_dict[index])
    result.append(tup)
  result.sort(key=operator.itemgetter(1),reverse=True)
  print(result[:k])




just_missed_list = knn_tester2(small_titanic,10,usable_columns,target)

[(91, 11), (51, 8), (2, 7), (76, 7), (5, 6), (21, 6), (64, 5), (17, 4), (23, 4), (26, 4)]


<h1>
Problem 5: Just missed was correct (15 points)
</h1>
<div class=h1_cell>
<p>
Let's delve a little deeper into the k+1 rows. I'd like to know how many times they were correct. If k=10 then count how many times the 11th row was correct. Store your result in missed_correct_list, in sorted fashion. Here are the first part of my results.
  <pre>
[(91, 11),
 (5, 6),
 (51, 6),
 (2, 5),
 (69, 4),
 (17, 3),
 (64, 3),
 (76, 3),
 (10, 2),
 (21, 2)]
 </pre>
  So when row 51 was in the top-10, it was correct 6 times.
</div>

In [82]:
cl = []
cd = {}
for i in range(len(small_titanic)):
  cd[i] = 0
  
for j,row in small_titanic.iterrows():
  actual = small_titanic.loc[target][j]
  top = knn2(j, small_titanic, usable_columns, 10, target)
  for item in top:
    pred = small_titanic.loc[target][j]
    if actual == pred:
      cd[j] += 1
      
for index in range(len(small_titanic)):
  correct_missed = (index, cd[index])
  cl.append(correct_missed)
  
cl.sort(key=operator.itemgetter(1), reverse=True)

0
(91, 0.0)
0
3
(54, 1.7320508075688772)
0
Braund, Mr. Owen Harris
(51, 1.0)
0
male
(40, 1.4142135623730951)
0
22.0
(76, 1.0)
0
1
(48, 1.4142135623730951)
0
0
(96, 1.4142135623730951)
0
A/5 21171
(69, 1.0)
0
7.25
(2, 1.4142135623730951)
0
nan
(2, 2.0)
0
S
(0, 1.4142135623730951)
0
0
(83, 1.7320508075688772)
0
22.0
(91, 0.0)
0
0
(76, 1.0)
0
0
(51, 1.0)
0
1
(21, 1.7320508075688772)
0
0
(51, 1.7320508075688772)
0
Child
(76, 1.4142135623730951)
0
1
(2, 1.4142135623730951)
0
0
(82, 1.4142135623730951)
0
0
(23, 1.4142135623730951)
0
0
(23, 1.4142135623730951)
0
1
(10, 1.7320508075688772)
0
0
(21, 1.4142135623730951)
0
0
(0, 1.4142135623730951)
0
0
(2, 1.4142135623730951)
0
1
(64, 1.4142135623730951)
0
0
(67, 1.4142135623730951)
0
[(12, 0.0), (37, 0.0), (51, 0.0), (59, 0.0), (67, 0.0), (69, 0.0), (75, 0.0), (80, 0.0), (86, 0.0), (89, 0.0)]
(26, 1.7320508075688772)
0


IndexError: ignored

In [0]:
correct_missed_list[:10]

[(91, 11),
 (5, 6),
 (51, 6),
 (2, 5),
 (69, 4),
 (17, 3),
 (64, 3),
 (76, 3),
 (10, 2),
 (21, 2)]

<h1>
Problem 6: Tip the scales (20 points)
</h1>
<div class=h1_cell>
<p>
Ok, finally to the question I really want to answer. If I am going to start including the 11th row by making k=11, I want to know if adding the row makes a difference. Would it have tipped the scales? As usual, go through all 891 rows and get the sorted distances for each. But now go ahead and get the votes of the top 10. Once you count the 10 votes, you will come up with a prediction as normal. What I want to know is whether the 11th vote would have tipped the scales. If you took just the 10, you get one result. If you add in the 11th vote you get a different result. Give me a sorted list of rows that tipped the scales acting as the 11th voter.
  <pre>
[(2, 4),
 (23, 3),
 (53, 2),
 (65, 1),
 (84, 1),
 (0, 0),
 (1, 0),
 (3, 0),
 (4, 0),
 (5, 0)]
 </pre>
  So row 2. acting as the k+1 voter, switched the outcome of the vote 4 times. Looking at this list, I would probably give up adding an 11th voter (i.e., changing k=11). Only a small number of examples where it actually makes a difference.
</div>

In [0]:
scale_tipper_list[:10]

[(2, 4),
 (23, 3),
 (53, 2),
 (65, 1),
 (84, 1),
 (0, 0),
 (1, 0),
 (3, 0),
 (4, 0),
 (5, 0)]